### Persistence time analysis

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import tensorflow as tf
from scipy.stats import norm
from sklearn.mixture import GaussianMixture as GMM
from scipy import stats
import json

In [9]:
def filter_data(data_in, max_step_size, window, restriction=10000):
    """
    Only keeps data for tracks that are long enough for hurst exponent estimation at a given downsampling step size.
    Args: 
        data_in: pandas dataframe, original data
        max_step_size: int, maximum downsampling step size
        window: int, size of rolling window for hurst component estimation
    Returns:
        filtered_data: pandas dataframe
    """
    tracks_to_keep = data_in.TrackID.value_counts().loc[lambda x: (x//max_step_size) > (window+1)].reset_index()['index']
    if restriction<len(tracks_to_keep):
        filtered_data = data_in[data_in['TrackID'].isin(tracks_to_keep[:restriction])]
    else:
        filtered_data = data_in[data_in['TrackID'].isin(tracks_to_keep)]
    return filtered_data

def downsample(data_input, down_int, start_index=0):
    
    data_out = data_input.iloc[start_index::down_int]
    return data_out



In [83]:
def get_mod(v):
    
    return np.sqrt(v.dot(v))

def get_step_vector(index,data_x,data_y,data_z):
    
    dx = data_x[index]-data_x[index-1]
    dy = data_y[index]-data_y[index-1]
    dz = data_z[index]-data_z[index-1]
    
    step_vector = np.array([dx,dy,dz])   

    return step_vector  

def get_cos_theta(data_in):
    
    track_id_values = np.unique(data['TrackID'])
    cos_theta_avg = []
    
    for tid in track_id_values:
        track_data = data[data['TrackID']==tid]
        x = track_data['Position X'].values
        y = track_data['Position Y'].values
        z = track_data['Position Z'].values
        
        length = len(track_data)
        #print(length)
        step_vectors = np.empty((length-1,3))
        cos_theta_temp = np.empty(length-2)
        for i in np.arange(length):
            if i>=1:
                
                step_vectors[i-1] = get_step_vector(i,x,y,z)
            if i>=2:
                
                v0 = step_vectors[i-2]
                v1 = step_vectors[i-1]
                cos_theta_temp[i-2] = np.dot(v0,v1)/(get_mod(v1)*get_mod(v0))
        
        print(len(cos_theta_temp))
        cos_theta_avg.append(np.nanmean(cos_theta_temp))
        
    return cos_theta_avg

def dsample_get_cos_theta(data_in, ds_rate):
    """
    Args:
        data_in: pandas dataframe
        ds_rate: int, downsampling step size
    Returns:
        h_arr: 2D np array
    """
    
    #if window % 2 == 1:  # for odd window size
     #   cos_theta_arr = np.empty((ds_rate,(len(data_in)//ds_rate)-(window)))
        
    #else:  # for even window size
     #   cos_theta_arr = np.empty((ds_rate,(len(data_in)//ds_rate)-(window+1)))
        
#    cos_theta_arr = np.empty(ds_rate,)
    for i in np.arange(ds_rate):
        
        downsampled_data = downsample(data_in, ds_rate, i)

        cos_theta_avgs = get_cos_theta(downsampled_data)
        print(len(cos_theta_avgs))
            
        if i == 0:
            cos_theta_arr = cos_theta_avgs#,dtype=object)
        else:
            
            if len(cos_theta_avgs)>np.shape(cos_theta_arr)[0]:
                cos_theta_avgs=cos_theta_avgs[:-1]
            
            cos_theta_arr.append(cos_theta_avgs)
        
    return cos_theta_arr

def get_cos_theta_dict(filtered_data, step_sizes):
    """
    """
    cos_theta_dict = {}
    
    for i, s in enumerate(step_sizes):
        cos_theta_arr = dsample_get_cos_theta(filtered_data, s)
        cos_theta_dict["{}".format(s)] = cos_theta_arr
        
    return cos_theta_dict

def save_cos_theta_data(cos_theta_data, file_name):#, window, step_sizes, restriction):
    """
    """
    #with open('cos_theta_dict_'+'w{}_'.format(window)+'s{}_'.format(max(step_sizes))+'r{}_'.format(restriction)+file_name, 'w')
    with open('cos_theta_dict_'+file_name, 'w') as file:
        file.write(json.dumps(cos_theta_data))
      

In [84]:
filenames = np.array(['Control_frame001-200',
                      'Control_frame200-400',
                      'Control_frame400-600',
                      'Control_frame600-800',
                      'Control_frame800-1000',
                      'Control_frame1000-1200',
                      'LanB1_frame001-200',
                      'LanB1_frame200-400',
                      'LanB1_frame400-600',
                      'LanB1_frame600-800',
                      'LanB1_frame800-1000',
                      'LanB1_frame1000-1200',
                      'defLanB1_300817_frame200-400',
                      'defLanB1_300817_frame400-600'])

In [86]:
step_sizes = [3,4]#,5,6,7,8]
restriction = 30
window = 20

# iterate over data files and save H dictionary 
for i, file in enumerate(filenames[6:7]):
    print('opened file {}'.format(file))
    data = pd.read_csv('haemocyte_tracking_data/' + file + '.csv')
    filtered_data = filter_data(data, max(step_sizes), window, restriction)
    cos_theta_dict = get_cos_theta_dict(filtered_data, step_sizes)
    #print(cos_theta_dict)
    save_cos_theta_data(cos_theta_dict, file)

opened file LanB1_frame001-200
198
195
18
5
9
123
155
8
3
42
13
50
28
3
4
20
24
66
3
14
0
26
23
2
2
27
3
9
42
7
129
5
127
1
38
26
34
101
169
98
109
4
6
0
121
0
4
54
100
23
27
24
46


<ipython-input-83-3dbd814d96e7>:41: RuntimeWarning: Mean of empty slice
  cos_theta_avg.append(np.nanmean(cos_theta_temp))
<ipython-input-83-3dbd814d96e7>:41: RuntimeWarning: Mean of empty slice
  cos_theta_avg.append(np.nanmean(cos_theta_temp))
<ipython-input-83-3dbd814d96e7>:41: RuntimeWarning: Mean of empty slice
  cos_theta_avg.append(np.nanmean(cos_theta_temp))
<ipython-input-83-3dbd814d96e7>:41: RuntimeWarning: Mean of empty slice
  cos_theta_avg.append(np.nanmean(cos_theta_temp))
<ipython-input-83-3dbd814d96e7>:41: RuntimeWarning: Mean of empty slice
  cos_theta_avg.append(np.nanmean(cos_theta_temp))


48
17
25
3
198
198
52
188
66
17
35
65
6
13
4
11
2
19
9
2
6
5
15
22
57
25
28
2
24
16
5
8
168
1
50
0
81
0
10
128
90
8
34
7
156
29
5
16
131
17
10
27
1
2
14
5
35
94
12
15
20
25
23
8
195
198
149
198
194
198
198
70
43
99
37
10
19
36
10
1
34
7
10
3
56
11
2
0
78
15
48
10
21
0
16
30
13
184
5
7
15
1
8
27
3
30
8
0
18
195
2
57


<ipython-input-83-3dbd814d96e7>:41: RuntimeWarning: Mean of empty slice
  cos_theta_avg.append(np.nanmean(cos_theta_temp))
<ipython-input-83-3dbd814d96e7>:41: RuntimeWarning: Mean of empty slice
  cos_theta_avg.append(np.nanmean(cos_theta_temp))
<ipython-input-83-3dbd814d96e7>:41: RuntimeWarning: Mean of empty slice
  cos_theta_avg.append(np.nanmean(cos_theta_temp))
<ipython-input-83-3dbd814d96e7>:41: RuntimeWarning: Mean of empty slice
  cos_theta_avg.append(np.nanmean(cos_theta_temp))
<ipython-input-83-3dbd814d96e7>:41: RuntimeWarning: Mean of empty slice
  cos_theta_avg.append(np.nanmean(cos_theta_temp))
<ipython-input-83-3dbd814d96e7>:41: RuntimeWarning: Mean of empty slice
  cos_theta_avg.append(np.nanmean(cos_theta_temp))
<ipython-input-83-3dbd814d96e7>:41: RuntimeWarning: Mean of empty slice
  cos_theta_avg.append(np.nanmean(cos_theta_temp))


15
66
74
7
7
0
19
20
18
8
7
81
1
27
9
4
6
7
150
6
3
1
34
139
39
15
25
9
91
6
195
4
6
10
8
25
196
0
5
23
3
70
33
48
56
5
29
68
66
44
18
1
29
0
27
0
112
198
5
5
64
6
137
16
196
23
5
20
83
2
81
134
17
2
49
22
30
71
15
34
20
144
32
28
63
79
46
53
41
21
0
41
21
3
107
79
53
1


...

<ipython-input-83-3dbd814d96e7>:41: RuntimeWarning: Mean of empty slice
  cos_theta_avg.append(np.nanmean(cos_theta_temp))
<ipython-input-83-3dbd814d96e7>:41: RuntimeWarning: Mean of empty slice
  cos_theta_avg.append(np.nanmean(cos_theta_temp))
<ipython-input-83-3dbd814d96e7>:41: RuntimeWarning: Mean of empty slice
  cos_theta_avg.append(np.nanmean(cos_theta_temp))
<ipython-input-83-3dbd814d96e7>:41: RuntimeWarning: Mean of empty slice
  cos_theta_avg.append(np.nanmean(cos_theta_temp))
<ipython-input-83-3dbd814d96e7>:41: RuntimeWarning: Mean of empty slice
  cos_theta_avg.append(np.nanmean(cos_theta_temp))
<ipython-input-83-3dbd814d96e7>:41: RuntimeWarning: Mean of empty slice
  cos_theta_avg.append(np.nanmean(cos_theta_temp))
<ipython-input-83-3dbd814d96e7>:41: RuntimeWarning: Mean of empty slice
  cos_theta_avg.append(np.nanmean(cos_theta_temp))
<ipython-input-83-3dbd814d96e7>:41: RuntimeWarning: Mean of empty slice
  cos_theta_avg.append(np.nanmean(cos_theta_temp))
<ipython-input-8

3
14
5
12
14
14
3
13
2
1
14
1
14
0
1
10
9
0
4
13
0
12
13
12
1
1
13
12
0
12
10
9
7
12
10
11
1
0
11
12
12
11
12
11
11
0
11
4
1
11
1
0
4
1
0
0
11
4
2
3
0
2
8
11
3
9
10
10
10
0
11
10
3
3
0
3
1
2
8
3
3
10
0
10
10
3
1
3
10
0
8
10
1
10
10
10
10
6
9
9
0
0
8
0
0
2
5
9
0
3
9
7
0
9
9
3
8
1
8
8
8
8
2
2
8
6
8
4
7
0
8
6
1
7
7
8
7
3
2
8
7
0
6
0
5
2
6
0
1
7
7
7
2
7
0
7
4
1
5
3
5
7
7
2
0
5
0
6
5
6
6
6
0
4
5
6
2
2
1
2
6
5
2
6
4
0
6
4
4
4
6
6
6
2
0
2
3
0
2
5
4
3
2
5
5
1
5
2
3
4
0
5
4
0
1
4
5


<ipython-input-83-3dbd814d96e7>:41: RuntimeWarning: Mean of empty slice
  cos_theta_avg.append(np.nanmean(cos_theta_temp))
<ipython-input-83-3dbd814d96e7>:41: RuntimeWarning: Mean of empty slice
  cos_theta_avg.append(np.nanmean(cos_theta_temp))
<ipython-input-83-3dbd814d96e7>:41: RuntimeWarning: Mean of empty slice
  cos_theta_avg.append(np.nanmean(cos_theta_temp))
<ipython-input-83-3dbd814d96e7>:41: RuntimeWarning: Mean of empty slice
  cos_theta_avg.append(np.nanmean(cos_theta_temp))
<ipython-input-83-3dbd814d96e7>:41: RuntimeWarning: Mean of empty slice
  cos_theta_avg.append(np.nanmean(cos_theta_temp))
<ipython-input-83-3dbd814d96e7>:41: RuntimeWarning: Mean of empty slice
  cos_theta_avg.append(np.nanmean(cos_theta_temp))
<ipython-input-83-3dbd814d96e7>:41: RuntimeWarning: Mean of empty slice
  cos_theta_avg.append(np.nanmean(cos_theta_temp))
<ipython-input-83-3dbd814d96e7>:41: RuntimeWarning: Mean of empty slice
  cos_theta_avg.append(np.nanmean(cos_theta_temp))
<ipython-input-8

5
5
1
4
4
4
4
4
2
4
4
3
4
0
4
0
3
4
4
0
4
2
3
3
3
3
3
2
3
3
3
3
0
2
2
3
3
3
3
0
3
1
3
2
2
2
0
2
0
2
1
2
2
2
2
0
2
2
2
1
1
1
0
0
0
1
1
1
1
1
0
1
1
0
1
1
1
0
0
0
0
0
0
0
0
0
0
0
0
4793


<ipython-input-83-3dbd814d96e7>:41: RuntimeWarning: Mean of empty slice
  cos_theta_avg.append(np.nanmean(cos_theta_temp))
<ipython-input-83-3dbd814d96e7>:41: RuntimeWarning: Mean of empty slice
  cos_theta_avg.append(np.nanmean(cos_theta_temp))
<ipython-input-83-3dbd814d96e7>:41: RuntimeWarning: Mean of empty slice
  cos_theta_avg.append(np.nanmean(cos_theta_temp))
<ipython-input-83-3dbd814d96e7>:41: RuntimeWarning: Mean of empty slice
  cos_theta_avg.append(np.nanmean(cos_theta_temp))
<ipython-input-83-3dbd814d96e7>:41: RuntimeWarning: Mean of empty slice
  cos_theta_avg.append(np.nanmean(cos_theta_temp))
<ipython-input-83-3dbd814d96e7>:41: RuntimeWarning: Mean of empty slice
  cos_theta_avg.append(np.nanmean(cos_theta_temp))
<ipython-input-83-3dbd814d96e7>:41: RuntimeWarning: Mean of empty slice
  cos_theta_avg.append(np.nanmean(cos_theta_temp))
<ipython-input-83-3dbd814d96e7>:41: RuntimeWarning: Mean of empty slice
  cos_theta_avg.append(np.nanmean(cos_theta_temp))
<ipython-input-8